# 1．各特徴の内容を理解
### 【目的】
* 目的変数と説明変数を整理して、「**何を予測するのか**」「**どんな特徴があるのか**」を明確にする。<br>
  　⇒ その後の前処理・学習・予測の方向性に大きく関わるため。
* 予測時にリークとなる変数は除外候補として整理する。<br>
    ⇒ 退職後にしか分からない情報（例：退職理由など）が混ざると、モデルが“カンニング”したような不正確な予測をしてしまうため。
* 行数・列数を確認し、データの規模感を把握する。<br>
  ⇒ データ量によって前処理方法（例：欠損値補完・エンコーディング方法・学習時間など）が変わるため。
　また、行数が多ければモデル精度向上の余地が大きいこともわかる。<br>
### 【具体策】
1. df.columnsによって、どのような変数があるかを一覧確認
2. df["列名"].unique()によって、各特徴量の内容を把握
3. df.head() や df["変数"].unique() で中身の例をチェック
4. df.dtypesにより、型（数値型・文字列型など）を確認して前処理方針を決定

### 【結果】
#### ① 特徴量を確認
   ##### 〇目的変数（y）
   - STATUS　（"ACTIVE" or "TERMINATED"） ⇒退職しているかどうかを二値分類で予測する。
   ##### 〇説明変数（X）

  * **数値型**
  　age（年齢）
  　length_of_service（勤続年数）

  * **カテゴリ型**
  　department_name（部署）
  　job_title（職種）
  　gender_short`性別）
  　city_name（勤務都市）
  　BUSINESS_UNIT（事業部門）



#### ② 除外候補
- termreason_desc, termtype_desc | 退職理由そのもの → **リーク情報**                    
- STATUS_YEAR                      | 「いつのデータか」を示すため、将来情報を含む可能性 → **時系列的リーク** 
- city_name                        | 値の種類が多く、説明力が低くノイズになる可能性              
- EmployeeID                       | 単なる識別子のため、**予測に不要**                     


#### ③方向性検討

* 数値型が少なく、全体的に**カテゴリデータ中心**。
  　→ モデル前処理では OneHotEncoderの活用を想定。
* 推測として、退職に関係しそうな特徴として、「勤続年数」「部署」「職種」が考えられる。
* EDA（可視化）で偏りを確認し、除外候補の最終判断を行う。

In [9]:
import pandas as pd
import os

df = pd.read_csv("data/raw/MFG10YearTerminationData.csv")

In [10]:
df.shape

(49653, 18)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49653 entries, 0 to 49652
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   EmployeeID           49653 non-null  int64 
 1   recorddate_key       49653 non-null  object
 2   birthdate_key        49653 non-null  object
 3   orighiredate_key     49653 non-null  object
 4   terminationdate_key  49653 non-null  object
 5   age                  49653 non-null  int64 
 6   length_of_service    49653 non-null  int64 
 7   city_name            49653 non-null  object
 8   department_name      49653 non-null  object
 9   job_title            49653 non-null  object
 10  store_name           49653 non-null  int64 
 11  gender_short         49653 non-null  object
 12  gender_full          49653 non-null  object
 13  termreason_desc      49653 non-null  object
 14  termtype_desc        49653 non-null  object
 15  STATUS_YEAR          49653 non-null  int64 
 16  STAT

In [13]:
df.isnull().sum()

EmployeeID             0
recorddate_key         0
birthdate_key          0
orighiredate_key       0
terminationdate_key    0
age                    0
length_of_service      0
city_name              0
department_name        0
job_title              0
store_name             0
gender_short           0
gender_full            0
termreason_desc        0
termtype_desc          0
STATUS_YEAR            0
STATUS                 0
BUSINESS_UNIT          0
dtype: int64

In [14]:
#ざっくりとしたデータの雰囲気つかむ
df.head()

,EmployeeID,recorddate_key,birthdate_key,orighiredate_key,terminationdate_key,age,length_of_service,city_name,department_name,job_title,store_name,gender_short,gender_full,termreason_desc,termtype_desc,STATUS_YEAR,STATUS,BUSINESS_UNIT
0,1318,12/31/2006 0:00,1/3/1954,8/28/1989,1/1/1900,52,17,Vancouver,Executive,CEO,35,M,Male,Not Applicable,Not Applicable,2006,ACTIVE,HEADOFFICE
1,1318,12/31/2007 0:00,1/3/1954,8/28/1989,1/1/1900,53,18,Vancouver,Executive,CEO,35,M,Male,Not Applicable,Not Applicable,2007,ACTIVE,HEADOFFICE
2,1318,12/31/2008 0:00,1/3/1954,8/28/1989,1/1/1900,54,19,Vancouver,Executive,CEO,35,M,Male,Not Applicable,Not Applicable,2008,ACTIVE,HEADOFFICE
3,1318,12/31/2009 0:00,1/3/1954,8/28/1989,1/1/1900,55,20,Vancouver,Executive,CEO,35,M,Male,Not Applicable,Not Applicable,2009,ACTIVE,HEADOFFICE
4,1318,12/31/2010 0:00,1/3/1954,8/28/1989,1/1/1900,56,21,Vancouver,Executive,CEO,35,M,Male,Not Applicable,Not Applicable,2010,ACTIVE,HEADOFFICE


In [15]:
for col in df.columns:
    print(col, df[col].unique()[:10])  # 先頭10件だけ


EmployeeID [1318 1319 1320 1321 1322 1323 1325 1328 1329 1330]
recorddate_key ['12/31/2006 0:00' '12/31/2007 0:00' '12/31/2008 0:00' '12/31/2009 0:00'
 '12/31/2010 0:00' '12/31/2011 0:00' '12/31/2012 0:00' '12/31/2013 0:00'
 '12/31/2014 0:00' '12/31/2015 0:00']
birthdate_key ['1/3/1954' '1/3/1957' '1/2/1955' '1/2/1959' '1/9/1958' '1/9/1962'
 '1/13/1964' '1/17/1956' '1/23/1967' '1/25/1967']
orighiredate_key ['8/28/1989' '8/31/1989' '9/2/1989' '9/5/1989' '9/8/1989' '9/9/1989'
 '9/10/1989' '9/15/1989' '9/16/1989' '9/20/1989']
terminationdate_key ['1/1/1900' '2/16/2009' '2/19/2014' '2/19/2009' '2/21/2009' '2/24/2014'
 '2/25/2014' '3/4/2014' '3/4/2009' '3/9/2009']
age [52 53 54 55 56 57 58 59 60 61]
length_of_service [17 18 19 20 21 22 23 24 25 26]
city_name ['Vancouver' 'Terrace' 'Nanaimo' 'Nelson' 'Kelowna' 'Victoria' 'Kamloops'
 'Fort St John' 'Surrey' 'Vernon']
department_name ['Executive' 'Store Management' 'Meats' 'Recruitment' 'Training'
 'Labor Relations' 'HR Technology' 'Employee R

In [16]:
# 列の中身チェック（エンター押すたびに列変わる）

for col in df.columns:

    
    print(f"== {col} ==")
    print(df[col].unique())
    input("Enterで次へ")

== EmployeeID ==
[1318 1319 1320 ... 8223 8226 8264]
== recorddate_key ==
['12/31/2006 0:00' '12/31/2007 0:00' '12/31/2008 0:00' '12/31/2009 0:00'
 '12/31/2010 0:00' '12/31/2011 0:00' '12/31/2012 0:00' '12/31/2013 0:00'
 '12/31/2014 0:00' '12/31/2015 0:00' '2/1/2009 0:00' '2/1/2014 0:00'
 '3/1/2014 0:00' '3/1/2009 0:00' '9/1/2012 0:00' '4/1/2014 0:00'
 '4/1/2009 0:00' '5/1/2009 0:00' '5/1/2014 0:00' '12/1/2014 0:00'
 '6/1/2009 0:00' '6/1/2014 0:00' '7/1/2014 0:00' '7/1/2009 0:00'
 '8/1/2009 0:00' '8/1/2014 0:00' '12/1/2009 0:00' '9/1/2014 0:00'
 '9/1/2009 0:00' '10/1/2009 0:00' '10/1/2014 0:00' '11/1/2014 0:00'
 '11/1/2009 0:00' '7/1/2007 0:00' '1/1/2015 0:00' '1/1/2010 0:00'
 '12/1/2010 0:00' '2/1/2015 0:00' '2/1/2010 0:00' '3/1/2015 0:00'
 '3/1/2010 0:00' '4/1/2010 0:00' '4/1/2015 0:00' '5/1/2015 0:00'
 '5/1/2010 0:00' '6/1/2010 0:00' '6/1/2015 0:00' '7/1/2010 0:00'
 '7/1/2015 0:00' '12/1/2015 0:00' '3/1/2011 0:00' '8/1/2015 0:00'
 '9/1/2010 0:00' '9/1/2015 0:00' '10/1/2015 0:00' '10

In [ ]:
# データ型チェック
df.dtypes

EmployeeID              int64
recorddate_key         object
birthdate_key          object
orighiredate_key       object
terminationdate_key    object
age                     int64
length_of_service       int64
city_name              object
department_name        object
job_title              object
store_name              int64
gender_short           object
gender_full            object
termreason_desc        object
termtype_desc          object
STATUS_YEAR             int64
STATUS                 object
BUSINESS_UNIT          object
dtype: object

In [ ]:
# 数値データ確認
df.describe()

# 結論：
# 数値変数は5列と少ない。
# 連続型データは、年齢（19〜65歳）、勤続年数（0〜26年）、
# 店舗番号（1〜46）は数値ではあるが、実はカテゴリ。
# # EDA（棒グラフ・ヒストグラムなど）ではカテゴリ中心になる見込み。

,EmployeeID,age,length_of_service,store_name,STATUS_YEAR
count,49653.000000,49653.000000,49653.000000,49653.000000,49653.000000
mean,4859.495740,42.077035,10.434596,27.297605,2010.612612
std,1826.571142,12.427257,6.325286,13.514134,2.845577
min,1318.000000,19.000000,0.000000,1.000000,2006.000000
25%,3360.000000,31.000000,5.000000,16.000000,2008.000000
50%,5031.000000,42.000000,10.000000,28.000000,2011.000000
75%,6335.000000,53.000000,15.000000,42.000000,2013.000000
max,8336.000000,65.000000,26.000000,46.000000,2015.000000


In [17]:
df.describe(include ='all')

,EmployeeID,recorddate_key,birthdate_key,orighiredate_key,terminationdate_key,age,length_of_service,city_name,department_name,job_title,store_name,gender_short,gender_full,termreason_desc,termtype_desc,STATUS_YEAR,STATUS,BUSINESS_UNIT
count,49653.000000,49653,49653,49653,49653,49653.000000,49653.000000,49653,49653,49653,49653.000000,49653,49653,49653,49653,49653.000000,49653,49653
unique,NaN,130,5342,4415,1055,NaN,NaN,40,21,47,NaN,2,2,4,3,NaN,2,2
top,NaN,12/31/2013 0:00,4/27/1956,10/16/2005,1/1/1900,NaN,NaN,Vancouver,Meats,Meat Cutter,NaN,F,Female,Not Applicable,Not Applicable,NaN,ACTIVE,STORES
freq,NaN,5215,40,50,42450,NaN,NaN,11211,10269,9984,NaN,25898,25898,48168,48168,NaN,48168,49068
mean,4859.495740,NaN,NaN,NaN,NaN,42.077035,10.434596,NaN,NaN,NaN,27.297605,NaN,NaN,NaN,NaN,2010.612612,NaN,NaN
std,1826.571142,NaN,NaN,NaN,NaN,12.427257,6.325286,NaN,NaN,NaN,13.514134,NaN,NaN,NaN,NaN,2.845577,NaN,NaN
min,1318.000000,NaN,NaN,NaN,NaN,19.000000,0.000000,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,2006.000000,NaN,NaN
25%,3360.000000,NaN,NaN,NaN,NaN,31.000000,5.000000,NaN,NaN,NaN,16.000000,NaN,NaN,NaN,NaN,2008.000000,NaN,NaN
50%,5031.000000,NaN,NaN,NaN,NaN,42.000000,10.000000,NaN,NaN,NaN,28.000000,NaN,NaN,NaN,NaN,2011.000000,NaN,NaN
75%,6335.000000,NaN,NaN,NaN,NaN,53.000000,15.000000,NaN,NaN,NaN,42.000000,NaN,NaN,NaN,NaN,2013.000000,NaN,NaN


In [ ]:
#　型や中身を見た後に、df.nunique()でその列にどれくらい種類があるのかを見る
#   どの列がIDか、どの列が過度に少ないか、どの列がノイズか理解し、予測に使う使わないの判断根拠になる。

df.nunique().sort_values(ascending = False)


# | `EmployeeID`      | 6284  | 識別子（モデル不要）               |
# | `birthdate_key`   | 5342  | 誕生日（日付多すぎ）→ 年齢に変換済みなら不要  |
# | `department_name` | 21    | ちょうど良いカテゴリ数 → 使える        |
# | `job_title`       | 47    | やや多いが、TargetEncodingで使える |
# | `BUSINESS_UNIT`   | 2     | 二値カテゴリ → そのまま使える         |
# | `STATUS`          | 2     | 目的変数                |


# 結論：
# ユニーク数の多い列（EmployeeID, birthdate_key, orig_hiredate_key）は識別・日付要素なので予測に使わない。

EmployeeID             6284
birthdate_key          5342
orighiredate_key       4415
terminationdate_key    1055
recorddate_key          130
age                      47
job_title                47
store_name               46
city_name                40
length_of_service        27
department_name          21
STATUS_YEAR              10
termreason_desc           4
termtype_desc             3
gender_full               2
gender_short              2
STATUS                    2
BUSINESS_UNIT             2
dtype: int64